## Лабораторная №4 Преобразование данных в упорядоченную форму (tidy data)

Задание на 4 лабораторную: взять свои данные из первой лабы и выяснить, насколько они опрятные. Если не очень, попробовать привести к опрятному виду. Если уже опрятные – привести к "демонстрационному" виду (когда их удобно в табличном виде воспринимать).

In [2]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Моллюск (морское ушко)

Возраст морского ушка определяют, разрезая раковину через конус, окрашивая ее и подсчитывая количество колец под микроскопом.

Для предсказания возраста используются и другие измерения (которые легче получить). 

Столбцы:

- Sex - Пол (мужской, женский, ребенок). Категориальный признак.

- Length - Длина (самое длинный замер раковины). Вещественный тип.

- Diameter - Диаметр (замеряется перпендикулярно длине). Вещественный тип.

- Height - Высота (моллюск с панцирем). Вещественный тип.

- WholeWeight - Полный вес. Вещественный тип.

- ShuckedWeight - Вес моллюска (без раковины). Вещественный тип.

- VisceraWeight - Вес внутренностей моллюска (после "кровотечения"). Вещественный тип.

- ShellWeight - Вес корпуса (после сушки). Вещественный тип.

- Rings - Кольца (дает возраст в годах). Целочисленный тип.

In [236]:
df = pd.read_csv('abalone.csv', sep=',')
df_new = df.copy()
df.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Hadley в работе (http://vita.had.co.nz/papers/tidy-data.pdf) предлагает три основных принципа, определяющих, является ли набор данных упорядоченным. Упорядоченные (tidy) данные это: 

1. Каждая переменная формирует столбец.

2. Каждое наблюдение формирует строку.

3. Каждый тип наблюдения формирует таблицу.


Исходя из этого определения, данные являются вполне упорядочеными.


Однако можно привести к "демонстрационному" виду.

In [238]:
# поделим на условные возрастных группы
df_new['Age'] = df_new.apply(lambda row: round(row['Rings']/6 + 1), axis = 1)

df_new = df_new.replace({'Age':{1: '1 (baby)', 
                            2:'2 (child)', 
                            3:'3 (young)',
                            4:'4 (adult)', 
                            5:'5 (old)',
                            6:'6 (very_old)'}})

# убираем признак 'пол' и 'количество колец'
df_new = df_new.drop(columns = (['Sex']), axis = 1)

# Переименуем все столбцы, относящиеся к весу, чтобы столбцы, которые мы хотим "сгруппировать", одинаково начинались
df_new.rename(columns = {'WholeWeight': 'Weight_Whole', 
                     'ShuckedWeight': 'Weight_Shucked',
                     'VisceraWeight': 'Weight_Viscera',
                     'ShellWeight': 'Weight_Shell'}, inplace = True)

df_new

,Length,Diameter,Height,Weight_Whole,Weight_Shucked,Weight_Viscera,Weight_Shell,Rings,Age
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15,4 (adult)
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7,2 (child)
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9,2 (child)
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10,3 (young)
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7,2 (child)
...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11,3 (young)
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10,3 (young)
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9,2 (child)
4175,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10,3 (young)


In [240]:
# Групируем по количеству колец (все значения заменяем на среднее).
df_Rings = df_new.drop(columns = (['Age']), axis = 1).groupby(['Rings'], as_index = False).mean()

df_Rings.head()

,Rings,Length,Diameter,Height,Weight_Whole,Weight_Shucked,Weight_Viscera,Weight_Shell
0,1,0.075000,0.055000,0.010000,0.002000,0.001000,0.000500,0.001500
1,2,0.150000,0.100000,0.025000,0.015000,0.004500,0.004000,0.005000
2,3,0.176000,0.128667,0.041667,0.028400,0.011767,0.006267,0.008933
3,4,0.221491,0.161579,0.053947,0.059605,0.024719,0.012956,0.018000
4,5,0.285739,0.210696,0.069913,0.125500,0.061696,0.027330,0.036770


In [241]:
# Групируем по возрасту (все значения заменяем на среднее).
df_Age = df_new.drop(columns = (['Rings']), axis = 1).groupby(['Age'], as_index = False).mean()
df_Age.head()

,Age,Length,Diameter,Height,Weight_Whole,Weight_Shucked,Weight_Viscera,Weight_Shell
0,1 (baby),0.112500,0.077500,0.017500,0.008500,0.002750,0.002250,0.003250
1,2 (child),0.462703,0.355709,0.118567,0.570855,0.260816,0.124520,0.159384
2,3 (young),0.584866,0.458885,0.158742,1.074457,0.464335,0.236330,0.307299
3,4 (adult),0.590556,0.468260,0.168991,1.147075,0.432284,0.240392,0.369873
4,5 (old),0.604737,0.477632,0.184211,1.259053,0.422947,0.246342,0.411842


In [242]:
# попробуем по весу cортировать
stubs = ['Weight']
df_Weight = pd.wide_to_long(df_Age.drop(columns = ['Length', 'Height','Diameter'], axis = 1),
                            stubnames = stubs,
                            i = ['Age'],
                            j = 'Weight_Type',
                            suffix='.+',
                            sep='_').reset_index()
df_Weight

,Age,Weight_Type,Weight
0,1 (baby),Whole,0.008500
1,2 (child),Whole,0.570855
2,3 (young),Whole,1.074457
3,4 (adult),Whole,1.147075
4,5 (old),Whole,1.259053
5,6 (very_old),Whole,1.734500
6,1 (baby),Shucked,0.002750
7,2 (child),Shucked,0.260816
8,3 (young),Shucked,0.464335
9,4 (adult),Shucked,0.432284


In [243]:
df_Weight1 = df_Weight.groupby(['Age', 'Weight_Type'])['Weight'].mean()
df_Weight1

Age           Weight_Type
1 (baby)      Shell          0.003250
              Shucked        0.002750
              Viscera        0.002250
              Whole          0.008500
2 (child)     Shell          0.159384
              Shucked        0.260816
              Viscera        0.124520
              Whole          0.570855
3 (young)     Shell          0.307299
              Shucked        0.464335
              Viscera        0.236330
              Whole          1.074457
4 (adult)     Shell          0.369873
              Shucked        0.432284
              Viscera        0.240392
              Whole          1.147075
5 (old)       Shell          0.411842
              Shucked        0.422947
              Viscera        0.246342
              Whole          1.259053
6 (very_old)  Shell          0.628333
              Shucked        0.594500
              Viscera        0.305833
              Whole          1.734500
Name: Weight, dtype: float64

In [244]:
df_Weight1.unstack('Weight_Type')

Weight_Type,Shell,Shucked,Viscera,Whole
Age,,,,
1 (baby),0.003250,0.002750,0.002250,0.008500
2 (child),0.159384,0.260816,0.124520,0.570855
3 (young),0.307299,0.464335,0.236330,1.074457
4 (adult),0.369873,0.432284,0.240392,1.147075
5 (old),0.411842,0.422947,0.246342,1.259053
6 (very_old),0.628333,0.594500,0.305833,1.734500


In [245]:
df_Weight1.unstack('Age')

Age,1 (baby),2 (child),3 (young),4 (adult),5 (old),6 (very_old)
Weight_Type,,,,,,
Shell,0.00325,0.159384,0.307299,0.369873,0.411842,0.628333
Shucked,0.00275,0.260816,0.464335,0.432284,0.422947,0.594500
Viscera,0.00225,0.124520,0.236330,0.240392,0.246342,0.305833
Whole,0.00850,0.570855,1.074457,1.147075,1.259053,1.734500


In [246]:
# пусть столбцов агрегаторов будет несколько

df_Weight2 = df_Weight.groupby(['Age', 'Weight_Type'])['Weight'].agg(['mean', 'max'])
df_Weight2

mean       max
Age          Weight_Type                    
1 (baby)     Shell        0.003250  0.003250
             Shucked      0.002750  0.002750
             Viscera      0.002250  0.002250
             Whole        0.008500  0.008500
2 (child)    Shell        0.159384  0.159384
             Shucked      0.260816  0.260816
             Viscera      0.124520  0.124520
             Whole        0.570855  0.570855
3 (young)    Shell        0.307299  0.307299
             Shucked      0.464335  0.464335
             Viscera      0.236330  0.236330
             Whole        1.074457  1.074457
4 (adult)    Shell        0.369873  0.369873
             Shucked      0.432284  0.432284
             Viscera      0.240392  0.240392
             Whole        1.147075  1.147075
5 (old)      Shell        0.411842  0.411842
             Shucked      0.422947  0.422947
             Viscera      0.246342  0.246342
             Whole        1.259053  1.259053
6 (very_old) Shell        0.628333  0.628333
             Shucked      0.594500  0.594500
             Viscera      0.305833  0.305833
             Whole        1.734500  1.734500

In [247]:
df_Weight2.unstack('Weight_Type')

mean                                     max            \
Weight_Type      Shell   Shucked   Viscera     Whole     Shell   Shucked   
Age                                                                        
1 (baby)      0.003250  0.002750  0.002250  0.008500  0.003250  0.002750   
2 (child)     0.159384  0.260816  0.124520  0.570855  0.159384  0.260816   
3 (young)     0.307299  0.464335  0.236330  1.074457  0.307299  0.464335   
4 (adult)     0.369873  0.432284  0.240392  1.147075  0.369873  0.432284   
5 (old)       0.411842  0.422947  0.246342  1.259053  0.411842  0.422947   
6 (very_old)  0.628333  0.594500  0.305833  1.734500  0.628333  0.594500   

                                  
Weight_Type    Viscera     Whole  
Age                               
1 (baby)      0.002250  0.008500  
2 (child)     0.124520  0.570855  
3 (young)     0.236330  1.074457  
4 (adult)     0.240392  1.147075  
5 (old)       0.246342  1.259053  
6 (very_old)  0.305833  1.734500